<div align="center">
    <h1>Escuela Politécnica Nacional</h1>
    <h3>Recuperación de información</h3>
</div>


*Taller 05:* Herramientas para indexación  


*Integrantes:*  
- Vickiann Jiménez
- Gabriela Salazar  
- Jostin Vega  

## Parte 1: Construcción manual de un índice invertido

In [1]:
!pip install whoosh
!pip install elasticsearch

In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
from collections import defaultdict
from whoosh import index
from whoosh.fields import Schema, TEXT
import os
from whoosh.qparser import QueryParser
from tqdm import tqdm
from elasticsearch import Elasticsearch
from elasticsearch import helpers

#### Paso 1: Cargar los Datos en Python

In [3]:
# Cargar el dataset desde el archivo CSV 
df = pd.read_csv('../data/wiki_movie_plots_deduped.csv')
# Mostrar la cantidad total de películas en el dataset
print(f"Total de películas en el dataset: {len(df)}")
# Filtrar las columnas Title y Plot
df = df[['Title', 'Plot']]
# Mostrar las primeras filas del DataFrame
print(df.head())


Total de películas en el dataset: 34886
                              Title  \
0            Kansas Saloon Smashers   
1     Love by the Light of the Moon   
2           The Martyred Presidents   
3  Terrible Teddy, the Grizzly King   
4            Jack and the Beanstalk   

                                                Plot  
0  A bartender is working at a saloon, serving dr...  
1  The moon, painted with a smiling face hangs ov...  
2  The film, just over a minute long, is composed...  
3  Lasting just 61 seconds and consisting of two ...  
4  The earliest known adaptation of the classic f...  


#### Paso 2: Normalización del texto

In [4]:
# Descargar el paquete de stopwords de nltk
nltk.download('stopwords')
# Lista de stopwords en inglés
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Función de normalización de texto con eliminación de stopwords
def normalize_text(text):
    """
    Limpia y normaliza el texto: convierte a minúsculas, elimina puntuación y stopwords.

    Parámetros:
    text : str 
        Texto a procesar.
    
    Retorno:
    list: Palabras relevantes del texto.
    """
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'[^\w\s]', '', text)  # Eliminar puntuación
    words = text.split()  # Tokenizar en palabras
    return [word for word in words if word not in stop_words]  # Eliminar stopwords


#### Paso 3: Construir el índice invertido

In [6]:
# Crear índice invertido 
inverted_index = defaultdict(list)

total_rows = len(df)
print(f"Total de documentos a procesar: {total_rows}")

for idx, row in enumerate(df.itertuples(index=True, name=None)):
    plot = row[2]  # El segundo elemento es la trama del `itertuples`
    words = normalize_text(plot)
    for word in set(words):  # Evitar duplicados de palabras en la misma trama
        inverted_index[word].append(idx)

    # Mostrar progreso cada 5000 documentos
    if (idx + 1) % 5000 == 0:
        print(f"{idx + 1} de {total_rows} documentos procesados...")

print("\nÍndice invertido construido con éxito.")

Total de documentos a procesar: 34886
5000 de 34886 documentos procesados...
10000 de 34886 documentos procesados...
15000 de 34886 documentos procesados...
20000 de 34886 documentos procesados...
25000 de 34886 documentos procesados...
30000 de 34886 documentos procesados...

Índice invertido construido con éxito.


In [7]:
# Mostrar una parte del índice invertido
print("\nEjemplo del índice invertido (primeras 5 palabras):")
for word, docs in list(inverted_index.items())[:5]:
    print(f"{word}: {docs}\n")


Ejemplo del índice invertido (primeras 5 palabras):
mans: [0, 32, 61, 69, 87, 88, 104, 178, 212, 220, 344, 366, 381, 394, 410, 455, 478, 522, 558, 577, 607, 660, 666, 763, 864, 867, 872, 1066, 1131, 1138, 1270, 1289, 1290, 1356, 1377, 1378, 1414, 1432, 1535, 1539, 1564, 1611, 1614, 1619, 1721, 1778, 1789, 1827, 1864, 1897, 1944, 1954, 1981, 1990, 1995, 2104, 2186, 2199, 2318, 2351, 2353, 2404, 2421, 2495, 2629, 2656, 2675, 2711, 2727, 2739, 2744, 2761, 2816, 2842, 2889, 2916, 3022, 3047, 3173, 3185, 3209, 3248, 3301, 3333, 3339, 3382, 3414, 3461, 3614, 3616, 3731, 3805, 3838, 3853, 3855, 3860, 3944, 3956, 4001, 4032, 4098, 4155, 4169, 4171, 4181, 4186, 4211, 4268, 4270, 4282, 4293, 4302, 4329, 4351, 4354, 4433, 4445, 4526, 4533, 4536, 4541, 4562, 4577, 4589, 4595, 4622, 4675, 4728, 4779, 4785, 4851, 4866, 4971, 5026, 5027, 5068, 5074, 5084, 5094, 5103, 5106, 5137, 5193, 5239, 5253, 5278, 5297, 5417, 5468, 5505, 5527, 5547, 5715, 5717, 5860, 5889, 5908, 6044, 6089, 6097, 6112, 6135, 61

#### Paso 4: Implementar función de consulta en el índice invertido

In [8]:
def search_inverted_index(query, inverted_index, df):
    """
    Realiza una búsqueda en el índice invertido y muestra los documentos coincidentes.

    Parámetros:
    query : str
        Consulta de búsqueda ingresada por el usuario.
    inverted_index : dict
        Índice invertido con palabras como claves y listas de índices de documentos como valores.
    df : DataFrame
        DataFrame que contiene los títulos y las tramas de las películas.

    Retorno:
    list of dict
        Lista de diccionarios con los resultados de la búsqueda. 
        Cada diccionario contiene 'index', 'title' y 'plot' del documento encontrado.
    """
    query_words = normalize_text(query)  # Normalizar y tokenizar la consulta
    if not query_words:
        print("La consulta no contiene palabras clave.")
        return
    
    # Buscar los documentos que contienen la primera palabra
    results = set(inverted_index.get(query_words[0], []))
    
    # Intersección con los documentos que contienen las demás palabras
    for word in query_words[1:]:
        results.intersection_update(inverted_index.get(word, []))
    
    if results:
        # Mostrar índices de los documentos
        print(f"Se encontraron {len(results)} resultados para '{query}':\n")
        print(f"Índices de los documentos: {sorted(results)}\n")
        
        # Mostrar index, título y plot de cada documento
        for doc_id in results:
            title = df.iloc[doc_id]['Title']
            plot = df.iloc[doc_id]['Plot']
            print(f"Index: {doc_id}\nTitle: {title}\nPlot: {plot[:300]}...\n")  # Mostrar primeros 300 caracteres del plot
    else:
        print(f"No se encontraron resultados para '{query}'.")

# Prueba de consulta
search_inverted_index("cyborg", inverted_index, df)


Se encontraron 61 resultados para 'cyborg':

Índices de los documentos: [6982, 8074, 10133, 10144, 10159, 10298, 10812, 11152, 11240, 11849, 12170, 12407, 12838, 12989, 13046, 13581, 14080, 14172, 14466, 14765, 15752, 15754, 16582, 16691, 16748, 17071, 17196, 17621, 21326, 21607, 22367, 23402, 23638, 23660, 33016, 33017, 33018, 33050, 33064, 33072, 33102, 33175, 33191, 33200, 33228, 33366, 33387, 33397, 33407, 33408, 33435, 33489, 33502, 33521, 33613, 33656, 33716, 33944, 34262, 34279, 34434]

Index: 14080
Title: Jason X
Plot: In the year 2010, Jason Voorhees (Kane Hodder) is captured by the United States government and held at the Crystal Lake Research Facility. Government scientist Rowan LaFontaine (Lexa Doig) decides to place Jason in frozen stasis after several failed attempts to kill him. While Private Samuel Johnson...

Index: 33408
Title: Godzilla: Final Wars
Plot: Years after an initial attack of Tokyo in 1954, Godzilla is entrapped under ice in Antarctica after a battle with t

## Parte 2: Whoosh para indexación y recuperación

#### Paso 1: Instalar Whoosh

In [9]:
pip install whoosh

Note: you may need to restart the kernel to use updated packages.


#### Paso 2: Configurar el índice con Whoosh

In [10]:
# Definir el esquema con campos Title y Plot
schema = Schema(Title=TEXT(stored=True), Plot=TEXT(stored=True))

# Crear una carpeta para almacenar el índice
index_dir = "whoosh_index"
if not os.path.exists(index_dir):
    os.mkdir(index_dir)

# Crear el índice Whoosh
ix = index.create_in(index_dir, schema)
print("Índice creado con éxito.")


Índice creado con éxito.


#### Paso 3: Agregar documentos al índice

In [11]:
# Función para indexar los documentos en lotes utilizando Whoosh
def whoosh_batch_index(ix, df, batch_size=500):
    """
    Indexa documentos en lotes utilizando Whoosh para mejorar la eficiencia.

    Parámetros:
    ----------
    ix : whoosh.index.Index
        Índice de Whoosh donde se guardan los documentos.
    df : DataFrame
        DataFrame que contiene las columnas 'Title' y 'Plot' con los datos a indexar.
    batch_size : int, opcional
        Tamaño del lote de documentos que se procesan juntos (por defecto 500).
    """
    batch = []  # Lista temporal para almacenar el lote de documentos
    with ix.writer() as writer:  # Se utiliza el writer() de Whoosh para agregar documentos al índice
        for idx, row in tqdm(df.iterrows(), total=len(df)):  # tqdm para mostrar el progreso
            batch.append({
                "Title": row["Title"],
                "Plot": row["Plot"]
            })

            if len(batch) >= batch_size:
                # Agregar los documentos del lote al índice
                for doc in batch:
                    writer.add_document(Title=doc["Title"], Plot=doc["Plot"])
                batch = []  # Limpiar el lote después de agregarlo al índice

        # Agregar los documentos restantes (si el total no es múltiplo del batch_size)
        if batch:
            for doc in batch:
                writer.add_document(Title=doc["Title"], Plot=doc["Plot"])

    print("Documentos indexados con éxito.")


# Ejecutar la indexación en lotes
whoosh_batch_index(ix, df, batch_size=1000)  


100%|██████████| 34886/34886 [01:03<00:00, 548.95it/s]


Documentos indexados con éxito.


#### Paso 4: Consultas en el índice Whoosh

In [12]:
def search_whoosh(query):
    """
    Realiza una búsqueda en el índice Whoosh y muestra los documentos coincidentes.

    Parámetros:
    query : str
        Término o frase de búsqueda ingresada por el usuario.
    """
    # Abrir el índice
    with ix.searcher() as searcher:
        # Crear un parser para el campo "Plot"
        query_parser = QueryParser("Plot", ix.schema)
        parsed_query = query_parser.parse(query)

        # Realizar búsqueda sin límite y obtener todos los resultados
        results = searcher.search(parsed_query, limit=None)
        if results:
            print(f"Se encontraron {len(results)} resultados para '{query}':\n")
            
            # Ordenar los resultados por índice de documento
            sorted_results = sorted(results, key=lambda x: x.docnum)
            indices = [result.docnum for result in sorted_results]
            print(f"Índices de los documentos encontrados: {indices}\n")

            # Mostrar los primeros 10 resultados en orden
            print("Mostrando los primeros 10 resultados:\n")
            for idx, result in enumerate(sorted_results[:10]):
                print(f"{idx + 1}. Index: {result.docnum}")
                print(f"Title: {result['Title']}\nPlot: {result['Plot'][:300]}...\n")  # Mostrar primeros 300 caracteres del plot
        else:
            print(f"No se encontraron resultados para '{query}'.")


In [13]:
# Prueba de consultas
search_whoosh("cyborg")

Se encontraron 62 resultados para 'cyborg':

Índices de los documentos encontrados: [6982, 8074, 10102, 10133, 10144, 10159, 10298, 10812, 11152, 11240, 11849, 12170, 12407, 12838, 12989, 13046, 13581, 14080, 14172, 14466, 14765, 15752, 15754, 16582, 16691, 16748, 17071, 17196, 17621, 21326, 21607, 22367, 23402, 23638, 23660, 33016, 33017, 33018, 33050, 33064, 33072, 33102, 33175, 33191, 33200, 33228, 33366, 33387, 33397, 33407, 33408, 33435, 33489, 33502, 33521, 33613, 33656, 33716, 33944, 34262, 34279, 34434]

Mostrando los primeros 10 resultados:

1. Index: 6982
Title: The Colossus of New York
Plot: Jeremy Spensser (Ross Martin), the brilliant young son of a New York family of scientists and humanitarians, is killed when hit by a truck as he chases his son's toy airplane. His death occurs on the eve of his winning the "International Peace Prize", and he leaves behind a wife (Mala Powers) and yo...

2. Index: 8074
Title: Cyborg 2087
Plot: Garth A7 (Michael Rennie), a cyborg from the 

## Parte 3: Usar Elasticsearch para Indexación y Recuperación

### Paso 1. Iniciar Elasticsearch con Docker

In [14]:
# docker pull elasticsearch:8.10.1
# docker run -d -p 9200:9200 -e "discovery.type=single-node" -e "xpack.security.enabled=false" --name elasticsearch-container elasticsearch:8.10.1

### Paso 2. Configurar el Cliente Elasticsearch en Python

In [15]:
pip install elasticsearch


Note: you may need to restart the kernel to use updated packages.


Crear una conexión y definir el esquema del índice:

In [16]:
# Conectar al servidor de Elasticsearch
es = Elasticsearch("http://localhost:9200")

# Comprobar la conexión
if es.ping():
    print("Conexión exitosa con Elasticsearch")
else:
    print("Error al conectar con Elasticsearch")


Conexión exitosa con Elasticsearch


### Paso 3: Crear el índice con los campos Title y Plot

In [17]:
index_name = "movies_index"

# Definir el mapeo del índice con los campos Title y Plot
mapping = {
    "mappings": {
        "properties": {
            "Title": {"type": "text"},
            "Plot": {"type": "text"}
        }
    }
}

# Crear el índice
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=mapping)
    print(f"Índice '{index_name}' creado con éxito.")
else:
    print(f"Índice '{index_name}' ya existe.")


Índice 'movies_index' ya existe.


### Paso 4: Indexar documentos en Elasticsearch

In [18]:
def elasticsearch_bulk_index(es, index_name, df, batch_size=500):
    """
    Indexa documentos en Elasticsearch utilizando la API `bulk`.

    Parámetros:
    ----------
    es : Elasticsearch
        Cliente de Elasticsearch conectado al servidor.
    index_name : str
        Nombre del índice de Elasticsearch donde se guardarán los documentos.
    df : DataFrame
        DataFrame que contiene las columnas 'Title' y 'Plot' con los datos a indexar.
    batch_size : int, opcional
        Tamaño del lote de documentos que se envían juntos (por defecto 500).

    """
    total_docs = len(df)  # Número total de documentos
    actions = []  # Lista de documentos a indexar
    progress_bar = tqdm(total=total_docs, desc="Indexando documentos")  # Barra de progreso

    for idx, row in df.iterrows():
        action = {
            "_index": index_name,
            "_id": idx,
            "_source": {
                "Title": row["Title"],
                "Plot": row["Plot"]
            }
        }
        actions.append(action)

        # Enviar lote cuando se alcance el tamaño batch_size
        if len(actions) >= batch_size:
            helpers.bulk(es, actions)
            progress_bar.update(len(actions))  # Actualizar la barra de progreso con la cantidad indexada
            actions = []  # Limpiar lista de acciones

    # Indexar los documentos restantes si los hay
    if actions:
        helpers.bulk(es, actions)
        progress_bar.update(len(actions))  # Actualizar barra con los documentos restantes

    progress_bar.close()
    print("Documentos indexados con éxito.")


In [19]:
elasticsearch_bulk_index(es, "movies_index", df)


Indexando documentos: 100%|██████████| 34886/34886 [00:14<00:00, 2437.91it/s]

Documentos indexados con éxito.


### Paso 5: Realizar consultas

In [20]:
def search_elasticsearch(query, max_results=1000):
    """
    Realiza una búsqueda en Elasticsearch y muestra los documentos coincidentes.

    Parámetros:
    ----------
    query : str
        Término o frase de búsqueda ingresada por el usuario.
    max_results : int
        Máximo número de documentos a imprimir (por defecto 100).
    """
    body = {
        "size": 10000,  # Puede recuperar hasta 10,000 documentos
        "query": {
            "match": {
                "Plot": query
            }
        }
    }

    # Realizar búsqueda
    response = es.search(index=index_name, body=body)
    hits = response['hits']['hits']

    if hits:
        print(f"Se encontraron {len(hits)} resultados para '{query}':\n")

        # Ordenar los resultados por el índice (ID) en orden ascendente
        sorted_hits = sorted(hits, key=lambda x: int(x['_id']))

        # Crear lista de índices
        indices = [int(hit['_id']) for hit in sorted_hits]

        print(f"Índices de los documentos encontrados (ordenados):\n")
        print(indices[:max_results])  # Mostrar los primeros max_results índices
        print(f"\nMostrando los primeros {min(max_results, len(sorted_hits))} documentos completos:\n")

        for hit in sorted_hits[:max_results]:  # Mostrar solo los primeros max_results resultados
            index = hit['_id']
            title = hit["_source"]["Title"]
            plot = hit["_source"]["Plot"][:300]  # Mostrar solo los primeros 300 caracteres del Plot
            print(f"Index: {index}\nTitle: {title}\nPlot: {plot}...\n")
        
        print(f"\nTotal de documentos encontrados: {len(hits)}")
    else:
        print(f"No se encontraron resultados para '{query}'.")


In [21]:
# Prueba de consulta con límite de resultados
search_elasticsearch("cyborg")

Se encontraron 62 resultados para 'cyborg':

Índices de los documentos encontrados (ordenados):

[6982, 8074, 10102, 10133, 10144, 10159, 10298, 10812, 11152, 11240, 11849, 12170, 12407, 12838, 12989, 13046, 13581, 14080, 14172, 14466, 14765, 15752, 15754, 16582, 16691, 16748, 17071, 17196, 17621, 21326, 21607, 22367, 23402, 23638, 23660, 33016, 33017, 33018, 33050, 33064, 33072, 33102, 33175, 33191, 33200, 33228, 33366, 33387, 33397, 33407, 33408, 33435, 33489, 33502, 33521, 33613, 33656, 33716, 33944, 34262, 34279, 34434]

Mostrando los primeros 62 documentos completos:

Index: 6982
Title: The Colossus of New York
Plot: Jeremy Spensser (Ross Martin), the brilliant young son of a New York family of scientists and humanitarians, is killed when hit by a truck as he chases his son's toy airplane. His death occurs on the eve of his winning the "International Peace Prize", and he leaves behind a wife (Mala Powers) and yo...

Index: 8074
Title: Cyborg 2087
Plot: Garth A7 (Michael Rennie), a